In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt


In [2]:

################ Method One ######### 

vid = cv2.VideoCapture('http://192.168.1.7:4747/mjpegfeed') 
eye_cascade = cv2.CascadeClassifier('x.xml')
font = cv2.FONT_HERSHEY_SIMPLEX
counter=0 # to detect sleep or not 
while(True): 
      
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read()    
 
    if(type(frame)!=None): # Async function
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        eyes = eye_cascade.detectMultiScale( img,scaleFactor = 1.1, minNeighbors = 5)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(img,(ex,ey),(ex+ew,ey+eh),(255,0,0),5)

        if(eyes==()):
            counter+=1
            if(counter>7):
                cv2.putText(img,  
                    'Sleep',  
                    (50, 250),  
                    font, 3,  
                    (255, 255, 255),  
                    2,  
                    cv2.LINE_4) 
        else:
            counter=0
    # Display the resulting frame 
    cv2.imshow('frame', img) 
      
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 

/home/adel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [3]:
##### METHOD TWO ########

from imutils.video import VideoStream
from imutils import face_utils
import datetime
import argparse
import imutils
import time
import dlib
import numpy as np
import cv2

from scipy.spatial import distance as dist



In [4]:
#### Global vars #####
EAR_AVG=0
Mouth_AVG=0

In [15]:
##### Test Phase ########

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear
 
# Initializing dlib's face detector (HOG-based)
# Creating the facial shape predictor using the 'shape_predictor_68_face_landmarks.dat' file
print("Initializing Facial Landmarking Predictor ->")
#Detecting the frontal faces
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Initializing the camera sensor to warm up
vid = cv2.VideoCapture('http://192.168.1.7:4747/mjpegfeed') 

blink_count = 0
total = 0

# Mouth
mouthDistance=[]

# Eyes
ears=[]

counter=0

# Looping over all the frames from the webcam stream
while True:
    # Grabbing the frames
	ret, frame = vid.read()
	if(type(frame)!=type(None)):
		# Setting the frame size to 500
		frame = imutils.resize(frame, width=500)
		gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
		cv2.putText(frame, 'Stare at the camera', (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

		# Detecting the faces from the frame
		rects = detector(frame, 0)
		counter+=1
		if(counter>150):
			break
		# Looping through each face detection
		for rect in rects:
			shape = predictor(gray, rect)
			shape = face_utils.shape_to_np(shape)

			leftEye = shape[lStart:lEnd]
			rightEye = shape[rStart:rEnd]
			leftEAR = eye_aspect_ratio(leftEye)
			rightEAR = eye_aspect_ratio(rightEye)


			##     Mouth
			distanceOfMouth=shape[67]-shape[63]
			if(distanceOfMouth[1]>0):
			    mouthDistance.append(distanceOfMouth[1])
			else: 
				mouthDistance.append(-1*distanceOfMouth[1])


			# average the eye aspect ratio together for both eyes
			ear = (leftEAR + rightEAR) / 2.0
			if(ear>0):
				ears.append(ear)
			else:
				ears.append(-1*ear)

				
			# Looping over the facial landmarks based on their (x,y) coordinates
			# Drawing a red circle over those landmarks
			for (x, y) in shape:
				cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
					
					
		# Showing the frame
		cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
 
	# Break from loop/end when the 'q' is pressed
	if key == ord("q"):
		break
        
# Destroying the windows once the frame display is over
cv2.destroyAllWindows()
vs.stop()
print(sum(mouthDistance)/len(mouthDistance))
print(sum(ears)/len(ears))
EAR_AVG=sum(ears)/len(ears)
Mouth_AVG=sum(mouthDistance)/len(mouthDistance)


Initializing Facial Landmarking Predictor ->
1.44516129032
0.233458600655


In [17]:
##### Second phase #####
 


(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear
 
# Initializing dlib's face detector (HOG-based)
# Creating the facial shape predictor using the 'shape_predictor_68_face_landmarks.dat' file
print("Initializing Facial Landmarking Predictor ->")
#Detecting the frontal faces
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Initializing the camera sensor to warm up
vid = cv2.VideoCapture('http://192.168.1.7:4747/mjpegfeed') 
time.sleep(2.0)

blink_count = 0
total = 0

# Mouth
mouthDistance=0
mouthCounter=0

eyeCounter=0
# Looping over all the frames from the webcam stream
while True:
    # Grabbing the frames
	ret, frame = vid.read()
	if(type(frame)!=type(None)):
		# Setting the frame size to 500
		frame = imutils.resize(frame, width=500)
		gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
		# cv2.putText(frame, "Blink Count = " + str(total), (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

		# Detecting the faces from the frame
		rects = detector(frame, 0)
		

		# Looping through each face detection
		for rect in rects:
			shape = predictor(gray, rect)
			shape = face_utils.shape_to_np(shape)

			leftEye = shape[lStart:lEnd]
			rightEye = shape[rStart:rEnd]
			leftEAR = eye_aspect_ratio(leftEye)
			rightEAR = eye_aspect_ratio(rightEye)


			##     Mouth
			mouthDistance=shape[67]-shape[63]
			if(mouthDistance[1]>Mouth_AVG):
				mouthCounter+=1
			else:
				mouthCounter=0
			if(mouthDistance[1]>Mouth_AVG and mouthCounter>20):
				cv2.putText(frame, "Yawn", (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

			# average the eye aspect ratio together for both eyes
			ear = (leftEAR + rightEAR) / 2.0

			leftEyeHull = cv2.convexHull(leftEye)
			rightEyeHull = cv2.convexHull(rightEye)
			# cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
			# cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
			if(ear<EAR_AVG):
				eyeCounter+=1
			else:
				eyeCounter=0
			if (ear < EAR_AVG and eyeCounter>20):
				cv2.putText(frame, "Sleeping", (0, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
	
			# otherwise, the eye aspect ratio is not below the blink
			# threshold


			# Looping over the facial landmarks based on their (x,y) coordinates
			# Drawing a red circle over those landmarks
			for (x, y) in shape:
				cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
					
					
		# Showing the frame
		cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
 
	# Break from loop/end when the 'q' is pressed
	if key == ord("q"):
		break
        
# Destroying the windows once the frame display is over
cv2.destroyAllWindows()
vs.stop()


Initializing Facial Landmarking Predictor ->
